This is a notebok that drafts a small script to pull in all the data for the WGS pathogen detection and microbiome data. It needs to pull in the following:
    * sequence summary file from albacore
    * two blast output files (specific database and NCBI)
    * get the length of all porchoped reads
    * get processing (T/F) for porchoped and nanolyze
    * get this all into one data frame
    
    
    

In [1]:
import os
import pandas as pd
import subprocess as sub
from Bio import SeqIO
import argparse

In [2]:
#parser = argparse.ArgumentParser(description='''
#This is a notebok that drafts a small script to pull in all the data for the WGS pathogen detection and microbiome data. It needs to pull in the following:

#    * sequence summary file from albacore
#    * two blast output files (specific database and NCBI)
#    * get the length of all porchoped reads
#    * get processing (T/F) for porchoped and nanolyze
#    * get this all into one data frame
#''')

#parser.add_argument("BASEDIR", help="base folder, supposed to have all the sub folders processed by WGS script")

In [3]:
#args = parser.parse_args()

In [4]:
#lets define the base folder
BASEDIR = '/home/yiheng/data/Wagga_run1'
#BASEDIR = args.BASEDIR
#this will become the only flag of argparse

In [5]:
#write a quick check that looks for all the right folders
folder_list = 'basecalled_data  scripts  tracking  workspace'.split(' ')
for x in range(0,folder_list.count('')):
    folder_list.remove('')
#fix this test
if not set(os.listdir(os.path.abspath(BASEDIR))) == set (folder_list):
    'Print something=asdfasdf'

In [6]:
seq_df_headers = ['read_id','passes_filtering', 'sequence_length_template', 'mean_qscore_template',\
                  'barcode_arrangement', 'barcode_score', 'kit', 'variant']

In [7]:
#now get the sequencing_summary file
base_called_folder = os.path.join(BASEDIR, 'basecalled_data')
for thing in os.listdir(base_called_folder):
    if not os.path.isdir(os.path.join(base_called_folder, thing)):
        next
    else:
        seq_sum_file = os.path.join(base_called_folder, thing, 'sequencing_summary.txt')
        if not os.path.exists(seq_sum_file):
            print('No sequencing summary file. Please go check')
            continue
        seq_df = pd.read_csv(seq_sum_file, sep='\t')
        #capture the thing as the prefix of the fastq/fasta files in the barcode folders
        prefix = thing
        #might be a better way to only read in the wanted columns. Not subsetting afterwards.
        #please go check
        seq_df = seq_df.loc[:, seq_df_headers].copy()

        

In [8]:
#now get all the rgblast_output databases done 
rg_blast_df_file_list = []
nt_blast_df_file_list = []
workspace = os.path.join(BASEDIR, 'workspace')
folder_counter = 0
for folder in os.listdir(workspace):
    folder = os.path.join(workspace,folder)
    if not os.path.isdir(folder):
        next
    folder_counter += 1
    for file in os.listdir(folder):
        if not file.endswith('.rgblast_output') or not file.endswith('.ntblast_output'):
            next
        if file.endswith('.rgblast_output'):
            rg_blast_df_file_list.append(os.path.join(folder, file))
        if file.endswith('.ntblast_output'):
            nt_blast_df_file_list.append(os.path.join(folder, file))
    if not len(nt_blast_df_file_list) == len(rg_blast_df_file_list) == folder_counter:
        print('Not all barcode folders have all blast output files')
    #print(folder)

Not all barcode folders have all blast output files
Not all barcode folders have all blast output files
Not all barcode folders have all blast output files
Not all barcode folders have all blast output files
Not all barcode folders have all blast output files
Not all barcode folders have all blast output files
Not all barcode folders have all blast output files


In [7]:
#now get all the names of the reads that survived the lyzing
workspace = os.path.join(BASEDIR, 'workspace')
folder_counter = 0
nl_readid_list = []
for folder in os.listdir(workspace):
    folder_long = os.path.join(workspace,folder)
    if not os.path.isdir(folder_long):
        next
    folder_counter += 1
    for file in os.listdir(folder_long):
        lyzed_fastq = (os.path.join(folder_long, '%s.%s.fastq' % (prefix, folder )))
        if not os.path.exists(lyzed_fastq):
            print("No lyzed fastq file present.")
        elif file == '%s.%s.fastq' % (prefix, folder ):
            print(lyzed_fastq)
            lyzed_fastq_name = '%s.name.tmp' % lyzed_fastq
            #cmd = "grep '^@' %s | cut -c1-37 > %s"%\
            # (lyzed_fastq, lyzed_fastq_name)
            #cmd = "grep '^@' %s | cut -d:-f1 > %s"%\
             #(lyzed_fastq, lyzed_fastq_name)
            cmd = r"grep '^@' %s | sed -e 's/@\([a-zA-Z0-9-]\+\).*/\1/' > %s"%\
            (lyzed_fastq, lyzed_fastq_name)
            #print(cmd)
            sub.check_output(cmd, shell=True, stderr=sub.STDOUT)
            #sub.run(cmd.split(' '))
            nl_readid_list.append(lyzed_fastq_name)
        else:
            next

/home/yiheng/data/Wagga_run1/workspace/barcode02/Wagga_run1_albacore202.barcode02.fastq
/home/yiheng/data/Wagga_run1/workspace/barcode06/Wagga_run1_albacore202.barcode06.fastq
/home/yiheng/data/Wagga_run1/workspace/barcode05/Wagga_run1_albacore202.barcode05.fastq
/home/yiheng/data/Wagga_run1/workspace/barcode04/Wagga_run1_albacore202.barcode04.fastq
/home/yiheng/data/Wagga_run1/workspace/barcode03/Wagga_run1_albacore202.barcode03.fastq
/home/yiheng/data/Wagga_run1/workspace/barcode01/Wagga_run1_albacore202.barcode01.fastq


In [8]:
#now read in the tmp file of nanolyzed ids and add them to the dataframe as column
def get_read_ids(filename_list):
    #write a check that both _list and header is a list
    read_id_list = []
    for fn in filename_list:
        tmp_list = pd.read_csv(fn, sep='\t', header=None).loc[:,0].tolist()
        read_id_list= read_id_list + tmp_list
    return read_id_list

In [9]:
nl_survied_list = get_read_ids(nl_readid_list)

In [154]:
#do same for porechop + length
#this is pretty slow. May consider parallzing.
pc_length_dict = {}
porechop_survived_list = []
folder_counter = 0
for folder in os.listdir(workspace):
    folder_long = os.path.join(workspace,folder)
    if not os.path.isdir(folder_long):
        next
    folder_counter += 1
    porechoped_file = os.path.join(folder_long,'%s.chopped.%s.fastq'%(prefix, folder))
    if not os.path.exists(porechoped_file):
            print("Porechopped fastq missing for %s." % folder)
    else:
        for seq in SeqIO.parse(porechoped_file, 'fastq'):
            pc_length_dict[seq.id] = len(seq.seq)
            porechop_survived_list.append(seq.id)
    
#take for loop from above to loop over chopped files
#filename = '/home/yiheng/data/Wagga_run1/workspace/barcode01/Wagga_run1_albacore202.chopped.barcode01.fastq'
#for seq in SeqIO.parse(filename, 'fastq'):
#    pc_length_dict[seq.id] = len(seq.seq)

In [155]:
porechop_survived_single_list = [x.split('_')[0] for x in porechop_survived_list]

In [156]:
seq_df['pc_survived'] = seq_df['read_id'].isin(porechop_survived_single_list)

In [157]:
seq_df.head()

read_id  passes_filtering  \
0  596c6bb5-0386-4a68-8c82-65f9678d9a7e              True   
1  98054683-bd72-4c88-bee1-24a1a34c547e              True   
2  999fd16f-fea1-4e95-b1df-98b95a19f9bc             False   
3  a9b8e298-55fd-412a-9a38-2fcdfe53b3d7              True   
4  db401cca-ae14-41ca-b5b3-7730aa521a31              True   

   sequence_length_template  mean_qscore_template barcode_arrangement  \
0                       735                11.552           barcode03   
1                      3739                13.289           barcode02   
2                       800                 5.958        unclassified   
3                      1202                12.653           barcode04   
4                      1812                11.826           barcode04   

   barcode_score kit variant  pc_survived  nl_survived  
0      75.932205  BC    var1         True         True  
1      99.166664  BC    var2        False        False  
2      36.051281  NB    var1        False        False  
3      84.366669  BC    var2         True         True  
4      91.830505  BC    var1         True         True

In [158]:
#add the nanolyze survived column
seq_df['nl_survived'] = seq_df['read_id'].isin(nl_survied_list)
#make porchoped survived column

In [159]:
import numpy as np

In [179]:
blast_header = ['qseqid',
 'sseqid',
 'evalue',
 'bitscore',
 'length',
 'pident',
 'nident',
 'sgi',
 'sacc',
 'staxids',
 'scomnames',
'sskingdoms']
# original headers: qseqid sseqid evalue bitscore length pident nident sgi sacc staxids sscinames scomnames sskingdoms
def make_all_blast_df(_list, header, chopped_len_dict):
    df = pd.DataFrame()
    #write a check that both _list and header is a list
    for x in _list:
        tmp_df = pd.read_csv(x, sep='\t',header=None, names=header)
        first_column = tmp_df.columns[0]
        tmp_df['read_id'] = tmp_df[first_column].apply(lambda x: str(x).split('_')[0])
        tmp_df['read_length_pc'] = tmp_df[first_column].apply(lambda x: chopped_len_dict[x])
        df = pd.concat([df, tmp_df.iloc[:,[0,1,2,4,5,6,8,9,10,12,13]]])
        
    #now reduce the columns to what we want
    
    print(first_column)
    #df['read_id'] = df.iloc[:,0].str.split('_')[0]
    return df

In [180]:
nt_df = make_all_blast_df(nt_blast_df_file_list, [x +'_nt' for x in  blast_header], pc_length_dict)
rg_df = make_all_blast_df(rg_blast_df_file_list, [x +'_rg' for x in  blast_header], pc_length_dict)
#reduce column number of blast dataframe to what you want before you merge

qseqid_nt
qseqid_rg


In [181]:
#need to take care of porchop split reads checkin if second last character of string is _
#make a new column of the blast_df that has the initial read_id
#need to check how merge behaves when getting a doublcate of value in one df.
tmp_df = pd.merge(seq_df, rg_df,how='outer',left_on= 'read_id', right_on='read_id')
final_df = pd.merge(tmp_df, nt_df,how='outer',left_on= 'read_id', right_on='read_id')

In [187]:
final_df.iloc[:200,0:25]

read_id  passes_filtering  \
0    596c6bb5-0386-4a68-8c82-65f9678d9a7e              True   
1    98054683-bd72-4c88-bee1-24a1a34c547e              True   
2    999fd16f-fea1-4e95-b1df-98b95a19f9bc             False   
3    a9b8e298-55fd-412a-9a38-2fcdfe53b3d7              True   
4    db401cca-ae14-41ca-b5b3-7730aa521a31              True   
5    f89602cc-8b37-4e09-8a70-231ec6489cb7              True   
6    fb086996-0f82-4078-8bb5-69c221e7caf4             False   
7    3c47496c-7e39-422b-9604-ee0384aea011              True   
8    43d19353-a579-4fb9-8fea-ea5945102c69              True   
9    54565f7c-5a54-44bb-8602-51e945b134f1             False   
10   67677898-7b0e-4b69-864b-ad5cac2410b9              True   
11   774e48de-5c99-4f7b-ade5-244a9162bcb2              True   
12   7c87f99d-cfac-4fad-9c67-c619574c142b              True   
13   87bda7b9-1b44-442f-84e0-18ec9caa72b5              True   
14   8f4e338e-734e-468b-a2da-f4632237c726             False   
15   9699d58f-fb21-4a87-8e28-18d4e295d7d9              True   
16   96e514b5-447a-4ba9-aeb1-1552c111792d              True   
17   9c0d77ca-7547-4b39-b36d-fda89869c4d9              True   
18   a3be3de2-9f10-4d8f-a265-6dc0101ed52c              True   
19   a5256722-d59b-46a2-9010-59d8098791d1              True   
20   b4f6d2de-d515-4656-a2e6-6c9ee1b376cd             False   
21   ba325770-bf0a-489a-b067-8b8d4e44dff7             False   
22   faba9518-d893-46db-bc9c-1b386e91d475              True   
23   0f527905-0b20-49fb-a48d-60d34e9c8680              True   
24   2abe07c6-db9e-4f4f-9179-715661a3964d              True   
25   582408f0-cb87-4d74-8fb8-c4cb63383d83              True   
26   71e19f47-2858-4f7f-ad73-f71f495cd8ed              True   
27   85e80f71-5278-4c5e-9f6f-e9026ac24e54              True   
28   967f9013-d883-4dd5-93af-8eb6457643f0              True   
29   970ce372-bfb6-4a6f-8694-412e1ef556ae              True   
..                                    ...               ...   
170  7a1b7d4c-c2f6-4aca-881b-37118a5116e1              True   
171  86e97741-0b84-4da2-ac20-4dcc44e3d14c              True   
172  9376f2d5-208b-4387-ad1b-84236956195a              True   
173  9ce33dfa-bba5-4082-ab87-fb773a73dab1              True   
174  b771b2b2-1429-4288-a3be-bb3599c58d97              True   
175  c061f8f6-9df0-46b5-a41e-1b4885f9d981              True   
176  e07af5af-bce9-400c-b2d0-e3a817882810              True   
177  f3eb5fef-b251-4a4e-b684-43bcc165b777              True   
178  f5047400-59de-484c-a1c0-4ecf455ac98b              True   
179  ff647e50-7eb6-49c7-8fc7-83395eef0eca              True   
180  04b35ec8-2a64-4459-81ae-ca190c963ed6              True   
181  067e1c9b-9821-4b38-bf70-eb1e54529dea              True   
182  3fe494a3-a4d4-4387-ae31-821d66ba022d              True   
183  502f3dac-4c72-49aa-8b6b-abe657aa8f41              True   
184  6712be21-da10-4571-918e-bc3140deb10a              True   
185  724d0f9a-1472-430d-8b6c-5fb315000a48              True   
186  75b5f52d-f274-4520-b928-acc47a1f0852              True   
187  809bc78b-47b8-4d8d-b794-5cf3118c5eaf              True   
188  9335c4f1-411d-40a6-a98e-94d2c645378f              True   
189  f02bb962-ef4d-44a3-a650-b45985b2bf25              True   
190  168ede92-aeb9-4a53-af25-a22fa55836eb              True   
191  38d18378-7902-4b2b-acc6-e7817b6eb11b              True   
192  4a2732aa-c85f-4cb6-8cd0-8b84ec79f595              True   
193  5db101a5-5a5d-4038-9f33-5583426c79e5              True   
194  62d7e884-1b2e-417a-911f-f080cfad7444              True   
195  ab6a026f-1fb3-4d21-8aaf-6a16de8bcd20              True   
196  b9e0bde1-d7c3-4686-941d-bda332f54d03              True   
197  e6d45c5e-5798-46a3-9550-00f4666b8b6f              True   
198  0b02202d-2c4d-420d-a7f5-5750658d7449              True   
199  0f0c566d-5057-47ee-9565-301051b331b2              True   

     sequence_length_template  mean_qscore_template barcode_arrangement  \
0                         735                11.552 

In [ ]:
def length_column(x):
    if x in pc_length_dict.keys():
        return pc_length_dict[x]
    else:
        return np.nan

In [ ]:
#porchop length column once pc_length_dict is done
seq_df['pc_length'] = seq_df.read_id.apply(lambda x: length_column(x))

In [ ]:
seq_df['pc_length'].unique()

In [ ]:
#remove tmp files again
pc_length_dict

In [127]:
#need to take care of porchop split reads checkin if second last character of string is _
#make a new column of the blast_df that has the initial read_id
#need to check how merge behaves when getting a doublcate of value in one df.
test_df = pd.merge(seq_df, rg_df,how='outer',left_on= 'read_id', right_on='read_id')

In [139]:
test_2 = test_df[test_df['qseqid_rg'].str.contains('_') == True]

In [144]:
test_df.iloc[[992299,992300],:]

read_id  passes_filtering  \
992299  5d2390b3-162a-41c2-9520-4d6121bb0522              True   
992300  5d2390b3-162a-41c2-9520-4d6121bb0522              True   

        sequence_length_template  mean_qscore_template barcode_arrangement  \
992299                      3212                13.501           barcode03   
992300                      3212                13.501           barcode03   

        barcode_score kit variant  pc_survived  nl_survived       ...        \
992299      89.525421  BC    var1         True         True       ...         
992300      89.525421  BC    var1         True         True       ...         

       length_rg pident_rg  nident_rg  sgi_rg      sacc_rg  staxids_rg  \
992299    1145.0     87.86     1006.0     0.0  Wheat_90339         NaN   
992300    1766.0     89.75     1585.0     0.0  Wheat_19144         NaN   

        sscinames_rg  scomnames_rg sskingdoms_rg  read_length_pc  
992299           NaN           NaN           NaN          1130.0  
992300           NaN           NaN           NaN          1703.0  

[2 rows x 24 columns]

In [143]:
test_2.loc[:,['read_id', 'qseqid_rg']]

read_id  \
133     f45fce15-8806-4659-83e2-50a50d09ed00   
191     38d18378-7902-4b2b-acc6-e7817b6eb11b   
303     ff4ea012-5b19-47fe-b04a-cbb5780f243e   
335     c08248f7-2c33-41ad-a842-8b0f7f474509   
488     dac1c1d9-a18c-454b-b482-fb9f83eb7f86   
782     a8f7dcda-1169-4a80-b885-83ec289fa34a   
1008    7c291113-d1df-4cff-a043-7284c0997ea1   
1083    a9f25b75-cf0a-4ade-8cdb-75d09f3b2e5a   
1238    1bb0fccc-e1d3-428a-8df1-7854f947921b   
1581    77b50f15-7547-4aa7-8e97-702f95846290   
1582    77b50f15-7547-4aa7-8e97-702f95846290   
1731    af7f5880-6556-4217-8cbd-f0e5b4755be4   
1732    af7f5880-6556-4217-8cbd-f0e5b4755be4   
1775    fd4b4d0f-af78-45ab-ba33-f1376e042b9d   
2013    aa6200dd-648b-4da5-8143-2bb37dda9100   
2025    55dac31d-b073-44fc-a750-73d5a1798e1a   
2032    dde540ae-5e3a-4aa2-a531-9d380265873a   
2130    ad94f2ff-8f3c-4253-8c53-d29738a38165   
2537    900b18e5-4083-4a99-826e-153f5c76cb6e   
2538    900b18e5-4083-4a99-826e-153f5c76cb6e   
2561    130dd46c-0575-41e8-b60a-4490e129a029   
2769    052446b2-80ec-47b3-84ae-ceaa95b2293e   
2881    262bcb33-c7b5-4002-a2da-f3f16e86aa72   
2882    262bcb33-c7b5-4002-a2da-f3f16e86aa72   
2900    27ddd864-2b0b-46f5-ad97-507cf0ec1e1f   
3055    5efedb2a-3244-48db-96fa-bbc24109b9cf   
3068    6d10872d-c8fa-4650-8393-4c26df1b49e5   
3116    a5311285-c397-4971-9d6e-df4e37da089c   
3117    a5311285-c397-4971-9d6e-df4e37da089c   
3179    1143eecd-6328-41c0-b382-89557d256cb5   
...                                      ...   
990962  bce08bad-f60c-46f0-a42a-9608b90bf6e2   
991044  5f67ad3a-457f-4123-9efd-292bda7c3753   
991211  b63beaec-f531-4d04-9be2-a8663b0c3a34   
991313  58bf0b99-79b7-4950-9623-37928e0116bf   
991327  9cc36fca-e356-4b93-ba81-7dd98ea91526   
991407  bcde2f3a-2237-4f5c-9045-e01f0564086d   
991499  3ae1f738-7ed7-40d1-8e53-ba831f249851   
991675  90151f08-de34-44b7-a3cd-bea6ad5b47ee   
991733  08f6ffcc-1ff0-4208-9845-0e9c3e1b6bd1   
991885  414f7b74-96b2-41fb-966c-cf2db7987a43   
991963  f74e86ed-e0b6-4b6f-845b-5a01821aff5e   
992159  720ccef6-d82c-4aab-90c9-e1afad00f9b9   
992257  67fb8015-b50d-4e6b-9533-dbe0e24082cc   
992299  5d2390b3-162a-41c2-9520-4d6121bb0522   
992300  5d2390b3-162a-41c2-9520-4d6121bb0522   
992662  960cf72a-e9c2-4a85-bdd3-7bf0f613b32d   
992699  dafaea70-3d70-4008-87df-be9842b603fc   
992737  6458d8ca-1c30-47d8-8669-fa04c603abf2   
992738  6458d8ca-1c30-47d8-8669-fa04c603abf2   
992926  30f3332c-a6ef-49fb-aed7-b20701eca45b   
993042  3a69c20c-4478-4080-ae2b-080f27f62879   
993289  91672c9a-6d4b-4bd8-8f5c-c20c18253d19   
993439  bee72538-9ba9-412f-92a9-c2b656de05b3   
993453  a493fa86-e265-4963-95f1-6f4892c3bc2c   
993480  76c9492f-b5a8-498e-a5ec-69ce10685052   
993965  726115fc-7d69-4146-b9d5-ef36d5bb8264   
994099  440a6d0a-3453-4075-95f1-591fc39f9c83   
994116  71782292-9bc8-4b2a-b288-ca3c328cee7e   
994228  4ac015c3-80fe-474b-a95b-79961f4f6090   
994337  1f2a3fef-5a66-4710-ae24-2ab2308e6695   

                                     qseqid_rg  
133     f45fce15-8806-4659-83e2-50a50d09ed00_1  
191     38d18378-7902-4b2b-acc6-e7817b6eb11b_1  
303     ff4ea012-5b19-47fe-b04a-cbb5780f243e_1  
335     c08248f7-2c33-41ad-a842-8b0f7f474509_1  
488     dac1c1d9-a18c-454b-b482-fb9f83eb7f86_1  
782     a8f7dcda-1169-4a80-b885-83ec289fa34a_1  
1008    7c291113-d1df-4cff-a043-7284c0997ea1_1  
1083    a9f25b75-cf0a-4ade-8cdb-75d09f3b2e5a_1  
1238    1bb0fccc-e1d3-428a-8df1-7854f947921b_1  
1581    77b50f15-7547-4aa7-8e97-702f95846290_1  
1582    77b50f15-7547-4aa7-8e97-702f95846290_2  
1731    af7f5880-6556-4217-8cbd-f0e5b4755be4_1  
1732    af7f5880-6556-4217-8cbd-f0e5b4755be4_2  
1775    fd4b4d0f-af78-45ab-ba33-f1376e042b9d_1  
2013    aa6200dd-648b-4da5-8143-2bb37dda9100_1  
2025    55dac31d-b073-44fc-a750-73d5a1798e1a_1  
2032    dde540ae-5e3a-4aa2-a531-9d380265873a_1  
2130    ad94f2ff-8f3c-4253-8c53-d29738a38165_1  
2537    900b18e5-4083-4a99-826e-153f5c76cb6e_1  
2538    900b18e5-4083-4a99-826e-153f5c76cb6e_2  
2561    130dd46c-0575-41e8-b60a

In [ ]:
test_df[test_df.read_id=='f7e483ca-c6b8-46be-975f-3bc433b3b0f6']

In [ ]:
test_df.loc[:, ['read_id', 'qseqid_rg']]

In [ ]:
rg_df.columns

In [ ]:
#now get the sequencing_summary file
base_called_folder = os.path.join(BASEDIR, 'basecalled_data')
for thing in os.listdir(base_called_folder):
    if not os.path.isdir(os.path.join(base_called_folder, thing)):
        next
    else:
        seq_sum_file = os.path.join(base_called_folder, thing, 'sequencing_summary.txt')
        if not os.path.exists(seq_sum_file):
            print('No sequencing summary file. Please go check')
        seq_df = pd.read_csv(seq_sum_file, sep='\t')
        #might be a better way to only read in the wanted columns. Not subsetting afterwards.
        #please go check
        seq_df = seq_df.loc[:, seq_df_headers].copy()